# Imports

In [1]:
import ccxt
import pandas as pd
from datetime import datetime
import psycopg2
from dotenv import find_dotenv, dotenv_values
import tqdm

# Config

In [2]:
config = dotenv_values(find_dotenv())

user = config.get('USERNAME_PG')
password = config.get('PASSWORD_PG')

# Helper Functions

In [3]:
def insert_ohlc_data(db_params, table_name, exchange, symbol, ohlc_data):
    """
    Inserts OHLC data into a dynamically specified PostgreSQL table.

    Parameters:
    - db_params: A dictionary containing database connection parameters.
    - table_name: The name of the table to insert data into.
    - exchange: The name of the exchange (e.g., 'Binance').
    - symbol: The trading pair or symbol (e.g., 'BTCUSDT').
    - ohlc_data: The OHLC data from the Binance API.
    """
    # Connect to the PostgreSQL database
    conn = psycopg2.connect(**db_params)
    cur = conn.cursor()

    # SQL query to insert data, using safe string formatting for the table name
    insert_query = f"""
    INSERT IGNORE INTO {table_name} (exchange, symbol, datetime, open, high, low, close, base_volume, quote_volume, base_buy_volume, quote_buy_volume)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    ON CONFLICT (exchange, symbol, datetime) DO NOTHING;
    """

    for data in ohlc_data:
        # Convert timestamp to datetime
        datetime_val = datetime.utcfromtimestamp(int(data[0]) / 1000.0)

        # Prepare data tuple
        data_tuple = (
            exchange,
            symbol,
            datetime_val,
            data[1],  # open
            data[2],  # high
            data[3],  # low
            data[4],  # close
            data[5],  # base_volume
            data[7],  # quote_volume (index 6 is close time, so we use index 7)
            data[9],  # base_buy_volume
            data[10]  # quote_buy_volume
        )

        # Execute the insert query
        cur.execute(insert_query, data_tuple)

    # Commit the transaction
    conn.commit()

    # Close the cursor and connection
    cur.close()
    conn.close()

def fetch_historical_data(symbol):
    since = binance_exchange.parse8601('2017-01-01T00:00:00Z')  # Example start date
    all_candles = []
    
    while True:
        candles = binance_exchange.fapipublic_get_klines(params={'symbol':symbol, 'interval':'1m', 'limit':1500, 'startTime':since})
        if not candles:
            break
        since = int(candles[-1][0]) + 1  # Increment `since` to fetch the next batch
        all_candles.extend(candles)
        # print(f"Fetched {len(candles)} candles for {symbol}")
    
    return all_candles

# Get Data

## Create CCXT Client

In [4]:
binance_exchange = ccxt.binance({
    'enableRateLimit': True,
    'options': {
        'defaultType': 'future'  # or 'spot' depending on your interest
    }
})

## Get List of Symbols

In [5]:
exchange_info = binance_exchange.fapipublic_get_exchangeinfo()

In [6]:
exchange_info.keys()

dict_keys(['timezone', 'serverTime', 'futuresType', 'rateLimits', 'exchangeFilters', 'assets', 'symbols'])

In [7]:
exchange_info['rateLimits']

[{'rateLimitType': 'REQUEST_WEIGHT',
  'interval': 'MINUTE',
  'intervalNum': '1',
  'limit': '2400'},
 {'rateLimitType': 'ORDERS',
  'interval': 'MINUTE',
  'intervalNum': '1',
  'limit': '1200'},
 {'rateLimitType': 'ORDERS',
  'interval': 'SECOND',
  'intervalNum': '10',
  'limit': '300'}]

In [8]:
df_symbols = pd.DataFrame(exchange_info['symbols']).drop('filters', axis='columns')
df_symbols['status'].value_counts()

status
TRADING            293
SETTLING            15
PENDING_TRADING      1
Name: count, dtype: int64

In [9]:
df_symbols['contractType'].value_counts()

contractType
PERPETUAL          304
CURRENT_QUARTER      2
NEXT_QUARTER         2
                     1
Name: count, dtype: int64

In [10]:
binance_markets = binance_exchange.fetch_markets()

In [11]:
df_binance_markets = pd.DataFrame(binance_markets)

In [12]:
df_binance_markets['type'].value_counts()

type
spot      2649
swap       340
future      30
Name: count, dtype: int64

In [13]:
df_symbols.head()

,symbol,pair,contractType,deliveryDate,onboardDate,status,maintMarginPercent,requiredMarginPercent,baseAsset,quoteAsset,...,quotePrecision,underlyingType,underlyingSubType,settlePlan,triggerProtect,liquidationFee,marketTakeBound,maxMoveOrderLimit,orderTypes,timeInForce
0,BTCUSDT,BTCUSDT,PERPETUAL,4133404800000,1569398400000,TRADING,2.5000,5.0000,BTC,USDT,...,8,COIN,[PoW],0,0.0500,0.012500,0.05,10000,"[LIMIT, MARKET, STOP, STOP_MARKET, TAKE_PROFIT...","[GTC, IOC, FOK, GTX, GTD]"
1,ETHUSDT,ETHUSDT,PERPETUAL,4133404800000,1569398400000,TRADING,2.5000,5.0000,ETH,USDT,...,8,COIN,[Layer-1],0,0.0500,0.012500,0.05,10000,"[LIMIT, MARKET, STOP, STOP_MARKET, TAKE_PROFIT...","[GTC, IOC, FOK, GTX, GTD]"
2,BCHUSDT,BCHUSDT,PERPETUAL,4133404800000,1569398400000,TRADING,2.5000,5.0000,BCH,USDT,...,8,COIN,[PoW],0,0.0500,0.015000,0.05,10000,"[LIMIT, MARKET, STOP, STOP_MARKET, TAKE_PROFIT...","[GTC, IOC, FOK, GTX, GTD]"
3,XRPUSDT,XRPUSDT,PERPETUAL,4133404800000,1569398400000,TRADING,2.5000,5.0000,XRP,USDT,...,8,COIN,[Payment],0,0.0500,0.012500,0.05,10000,"[LIMIT, MARKET, STOP, STOP_MARKET, TAKE_PROFIT...","[GTC, IOC, FOK, GTX, GTD]"
4,EOSUSDT,EOSUSDT,PERPETUAL,4133404800000,1569398400000,TRADING,2.5000,5.0000,EOS,USDT,...,8,COIN,[Layer-1],0,0.1000,0.010000,0.10,10000,"[LIMIT, MARKET, STOP, STOP_MARKET, TAKE_PROFIT...","[GTC, IOC, FOK, GTX, GTD]"


## Retrieve OHLC Data and Write to Database

In [14]:
list_symbols = list(df_symbols['symbol'])

db_params = {
    'dbname': 'crypto_data',
    'user': user,
    'password': password,
    'host': 'localhost'
}
table_name = 'ohlc_1m_swaps_test'

for i, symbol in enumerate(list_symbols):
    print(f'{i+1}/{len(list_symbols)}: {symbol}', end='\r')
    try:
        ohlc_data = fetch_historical_data(symbol)
        insert_ohlc_data(db_params, table_name, 'binance', symbol, ohlc_data)
        print(f"Data inserted for {symbol}")
    except Exception as e:
        print(f"Error fetching or inserting data for {symbol}: {e}")

KeyboardInterrupt: 

# Asymc Stuff

In [ ]:
import asyncio
import aiohttp